In [2]:
!pip install scapy

  Using cached scapy-2.6.1-py3-none-any.whl.metadata (5.6 kB)
Using cached scapy-2.6.1-py3-none-any.whl (2.4 MB)


In [1]:
from scapy.all import rdpcap, IP, TCP, UDP, Raw
import pandas as pd

packets = rdpcap("sample_traffic.pcap")
data = []

for i, pkt in enumerate(packets):
    if IP in pkt:
        src_ip = pkt[IP].src
        dst_ip = pkt[IP].dst
        proto = "Other"
        dst_port = None
        src_port = None
        flags = None
        ttl = pkt[IP].ttl
        payload_len = len(pkt[Raw].load) if Raw in pkt else 0
        tcp_window = pkt[TCP].window if TCP in pkt else None
        tcp_flags_int = pkt[TCP].flags.value if TCP in pkt else None

        if TCP in pkt:
            proto = "TCP"
            dst_port = pkt[TCP].dport
            src_port = pkt[TCP].sport
            flags = pkt[TCP].flags
        elif UDP in pkt:
            proto = "UDP"
            dst_port = pkt[UDP].dport
            src_port = pkt[UDP].sport

        data.append({
            "src_ip": src_ip,
            "dst_ip": dst_ip,
            "src_port": src_port,
            "dst_port": dst_port,
            "protocol": proto,
            "packet_length": len(pkt),
            "payload_len": payload_len,
            "ttl": ttl,
            "tcp_flags": str(flags),
            "tcp_flags_int": tcp_flags_int,
            "tcp_window": tcp_window,
            "index": i  # for ordering
        })

df = pd.DataFrame(data)
df.to_csv("packets_enriched.csv", index=False)
print("✅ 已成功擷取更多封包特徵，存為 packets_enriched.csv")


✅ 已成功擷取更多封包特徵，存為 packets_enriched.csv
